In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('fake_news_dataset.csv')

In [6]:
df.drop(['date'],axis=1,inplace=True)

In [11]:
columns = df.drop(['title','text','label'],axis=1).columns

In [8]:
df = df.dropna()

In [9]:
df.drop(['author'],axis=1,inplace=True)

In [12]:
for column in columns:
    print(f"column:\n{df[column].value_counts()}")
    print(f"is_null:{df[column].isnull().sum()}\n")

column:
source
Daily News      2316
BBC             2280
CNN             2267
The Guardian    2254
Fox News        2251
NY Times        2242
Reuters         2239
Global Times    2196
Name: count, dtype: int64
is_null:0

column:
category
Health           2622
Sports           2612
Entertainment    2607
Technology       2599
Business         2576
Politics         2518
Science          2511
Name: count, dtype: int64
is_null:0



In [13]:
for column in columns:
    categories = df[column].unique()
    for category in categories:
        df[f"is_{category}"] = (df[column] == category).astype('int')

In [14]:
df.drop(['source','category'],axis=1,inplace=True)

In [15]:
df['label'] = (df['label'] == 'real').astype('int')

In [16]:
X = df.drop(['title','text','label'],axis=1)
y = df['label']

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [18]:
from sklearn.model_selection import train_test_split 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.3,random_state=42)

In [19]:
from keras import Sequential as sq 
from keras import layers 

In [20]:
model1 = sq([
    layers.Dense(units=1,input_shape=(X_train.shape[1],),activation='sigmoid')
])

/Users/erdaulet/Desktop/allProjects/myenv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [26]:
from keras import callbacks
early_stop = callbacks.EarlyStopping(monitor='accuracy',patience=10,restore_best_weights=True)

In [27]:
model1.fit(X_train,y_train,epochs=1000,batch_size=32,callbacks=[early_stop],validation_split=0.2)

Epoch 1/1000
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 845us/step - accuracy: 0.5142 - loss: 0.6927 - val_accuracy: 0.4966 - val_loss: 0.6943
Epoch 2/1000
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 870us/step - accuracy: 0.5134 - loss: 0.6929 - val_accuracy: 0.4978 - val_loss: 0.6942
Epoch 3/1000
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 876us/step - accuracy: 0.5128 - loss: 0.6929 - val_accuracy: 0.4986 - val_loss: 0.6942
Epoch 4/1000
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step - accuracy: 0.5098 - loss: 0.6926 - val_accuracy: 0.4986 - val_loss: 0.6943
Epoch 5/1000
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step - accuracy: 0.5128 - loss: 0.6929 - val_accuracy: 0.5018 - val_loss: 0.6939
Epoch 6/1000
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step - accuracy: 0.5146 - loss: 0.6932 - val_accuracy: 0.4958 - val_loss: 0.6941
Epoch 7/1000
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 771us/step - accuracy: 0.5176 - loss: 0.6922 - val_accuracy: 0.4994 - val_loss: 0.6939
Epoch 8/1000
316/316 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - accuracy: 0.5083 -

In [28]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report
model2 = RandomForestClassifier(n_estimators=100,class_weight='balanced')
model2.fit(X_train,y_train)
y_pred = model2.predict(X_test)
print(f"{classification_report(y_test,y_pred)}")

              precision    recall  f1-score   support

           0       0.50      0.53      0.52      2735
           1       0.49      0.46      0.47      2679

    accuracy                           0.50      5414
   macro avg       0.50      0.50      0.49      5414
weighted avg       0.50      0.50      0.49      5414



In [29]:
df

,title,text,label,is_NY Times,is_Fox News,is_CNN,is_Reuters,is_Daily News,is_Global Times,is_The Guardian,is_BBC,is_Politics,is_Business,is_Science,is_Technology,is_Health,is_Sports,is_Entertainment
0,Foreign Democrat final.,more tax development both store agreement lawy...,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,To offer down resource great point.,probably guess western behind likely next inve...,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Himself church myself carry.,them identify forward present success risk sev...,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
3,You unit its should.,phone which item yard Republican safe where po...,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,Billion believe employee summer how.,wonder myself fact difficult course forget exa...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,House party born.,hit and television I change very our happy doo...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
19996,Though nation people maybe price box.,fear most meet rock even sea value design stan...,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
19997,Yet exist with experience unit.,activity loss very provide eye west create wha...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
19998,School wide itself item.,term point general common training watch respo...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0


In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer as tfdV 
from nltk.corpus import stopwords
import nltk
import re
nltk.download('stopwords')
stop_words = stopwords.words('english')
def cleaner(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '',text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/erdaulet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
df['cleaned_title'] = df['title'].apply(cleaner)
df['cleaned_text'] = df['text'].apply(cleaner)

In [54]:
vectorizer = tfdV()
X = vectorizer.fit_transform(df['cleaned_text'])
y = df['label']

In [55]:
X = X.toarray()

In [56]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.3,random_state=42)

In [57]:
from sklearn.naive_bayes import MultinomialNB as MNB
model3 = MNB()
model3.fit(X_train,y_train)
y_pred = model3.predict(X_test)

In [58]:
from sklearn.metrics import classification_report 
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.51      0.68      0.58      2735
           1       0.50      0.33      0.39      2679

    accuracy                           0.50      5414
   macro avg       0.50      0.50      0.49      5414
weighted avg       0.50      0.50      0.49      5414

